## ECMWF API Authorization Review

This notebook uses the ECMWF python library to access its Web API and test various authentication conditions to see what responses are provided under various conditions. More information, including code samples and documentation, is available in the [ECMWF Wiki](https://software.ecmwf.int/wiki/display/WEBAPI/Access+ECMWF+Public+Datasets).

### ECMWF API Key Required

Note that users must be registered with ECMWF and provide a valid API key to access data using the API. Users can register with ECMWF at https://apps.ecmwf.int/registration/

### ECMWF Python Library

You can install the `ecmwfapi` python library by running:

`pip install https://software.ecmwf.int/wiki/download/attachments/56664858/ecmwf-api-client-python.tgz`

### Summary of Findings

* The ECMWF API requires a valid ECMWF API key, _which can only be acquired by registering with ECMWF on their website._
* For self-registered users, the corresponding email address associated with the ECMWF key must also be included. This could be problematic if their email address used with ECMWF differs from that used to register with Planet OS. These credentials should be stored as an independent pair, and not assume emails are identical across platforms.
* When attempting to retreive data, an APIException is raised when the ECMWF API key is invalid or missing.
* When attempting to retreive data, an APIException is raised when the user has not explicitly accepted the terms and conditions of the dataset. This may only be accomplished through the ECMWF UI, however the API does return a link to the license terms in the APIException response.

In [1]:
import os
from ecmwfapi import ECMWFDataServer

# Configure ECMWF access authentication. The library looks for
# these variables in the environment, or a .ecmwfapirc file in
# the user's home directory.

# Users must register with ECMWF to acquire an API key, this can
# be done at https://apps.ecmwf.int/registration/
os.environ["ECMWF_API_KEY"] = 'ECMWF_API_KEY'
os.environ["ECMWF_API_EMAIL"] = 'ECMWF_ACCOUNT_EMAIL_ADDRESS'
os.environ["ECMWF_API_URL"] = 'https://api.ecmwf.int/v1'

In [2]:
# Sample Request
# See https://software.ecmwf.int/wiki/display/WEBAPI/Python+ERA-interim+examples
server = ECMWFDataServer()
server.retrieve({
    'stream'    : "oper",
    'levtype'   : "sfc",
    'param'     : "165.128/166.128/167.128",
    'dataset'   : "interim",
    'step'      : "0",
    'grid'      : "0.75/0.75",
    'time'      : "00/06/12/18",
    'date'      : "2014-07-01/to/2014-07-02",
    'type'      : "an",
    'class'     : "ei",
    'format'    : "netcdf",
    'target'    : "interim_2014-07-01to2014-07-02_00061218.nc"
 })

2016-11-01 12:58:43 ECMWF API python library 1.4.1
2016-11-01 12:58:43 ECMWF API at https://api.ecmwf.int/v1
2016-11-01 12:58:44 Welcome Chris Kalima
2016-11-01 12:58:45 In case of problems, please check https://software.ecmwf.int/wiki/display/WEBAPI/Troubleshooting or contact calldesk@ecmwf.int
2016-11-01 12:58:45 You may also wish to check the MARS service activity page http://apps.ecmwf.int/mars-activity/
2016-11-01 12:58:46 Request submitted
2016-11-01 12:58:46 Request id: 5818f3f6f7414d50c0cb6ed5
2016-11-01 12:58:46 Request is queued
Calling '['nice', 'mars', '/tmp/tmp-_marsNdqCKJ.req']'
mars - WARN -
mars - WARN - From 9 February 2016 10AM (UTC) MARS uses versions of
mars - WARN - libemos newer than 4.3.0. For more details, see
mars - WARN - https://software.ecmwf.int/wiki/display/EMOS/Bug+fix+implemented+in+EMOSLIB+4.3.x
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20161101.195847 - Welcome to MARS with grib_api and ODB
mars - INFO   - 20161101.195847 - MARS Clie

In [3]:
# Let's invalidate the key and see what happens...
os.environ["ECMWF_API_KEY"] = 'badkey'

server = ECMWFDataServer()
server.retrieve({
    'stream'    : "oper",
    'levtype'   : "sfc",
    'param'     : "165.128/166.128/167.128",
    'dataset'   : "interim",
    'step'      : "0",
    'grid'      : "0.75/0.75",
    'time'      : "00/06/12/18",
    'date'      : "2014-07-01/to/2014-07-02",
    'type'      : "an",
    'class'     : "ei",
    'format'    : "netcdf",
    'target'    : "interim_2014-07-01to2014-07-02_00061218.nc"
 })

2016-11-01 12:59:07 ECMWF API python library 1.4.1
2016-11-01 12:59:07 ECMWF API at https://api.ecmwf.int/v1


APIException: 'ecmwf.API error 1: Invalid or missing key or token'

With an invalid key, an `APIException` is raised when a retrieval request is sent:

`APIException: 'ecmwf.API error 1: Invalid or missing key or token'`

In [4]:
# Let's try an invalid email with a valid API key...
os.environ["ECMWF_API_KEY"] = 'c73ca7414cebdf3a6c12b86b3b84cabd'
os.environ["ECMWF_API_EMAIL"] = 'bademail@planetos.com'

server = ECMWFDataServer()

print("Server Email: %s" % server.email)

server.retrieve({
    'stream'    : "oper",
    'levtype'   : "sfc",
    'param'     : "165.128/166.128/167.128",
    'dataset'   : "interim",
    'step'      : "0",
    'grid'      : "0.75/0.75",
    'time'      : "00/06/12/18",
    'date'      : "2014-07-01/to/2014-07-02",
    'type'      : "an",
    'class'     : "ei",
    'format'    : "netcdf",
    'target'    : "interim_2014-07-01to2014-07-02_00061218.nc"
 })

Server Email: bademail@planetos.com
2016-11-01 13:01:59 ECMWF API python library 1.4.1
2016-11-01 13:01:59 ECMWF API at https://api.ecmwf.int/v1
2016-11-01 13:02:00 Welcome Chris Kalima
2016-11-01 13:02:00 In case of problems, please check https://software.ecmwf.int/wiki/display/WEBAPI/Troubleshooting or contact calldesk@ecmwf.int
2016-11-01 13:02:00 You may also wish to check the MARS service activity page http://apps.ecmwf.int/mars-activity/
2016-11-01 13:02:01 Request submitted
2016-11-01 13:02:01 Request id: 5818f4b9ff29d051f0be4bde
2016-11-01 13:02:01 Request is queued
Calling '['nice', 'mars', '/tmp/tmp-_marsRUGb1D.req']'
mars - WARN -
mars - WARN - From 9 February 2016 10AM (UTC) MARS uses versions of
mars - WARN - libemos newer than 4.3.0. For more details, see
mars - WARN - https://software.ecmwf.int/wiki/display/EMOS/Bug+fix+implemented+in+EMOSLIB+4.3.x
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20161101.200202 - Welcome to MARS with grib_api and ODB
mars - 

Success! Seems like the email is not used to authenticate the request, only the API key. According to the [ECMWF REST API documentation](https://software.ecmwf.int/wiki/pages/viewpage.action?pageId=22907895) an email address is only required for "self-registered users." It's unclear at this time exactly what "self-registered" means.

In [5]:
# Finally, let's try to access a dataset that we have not explicitly
# agreed to the terms. We'll use the TIGGE dataset in this case,
# see https://software.ecmwf.int/wiki/display/WEBAPI/Python+TIGGE+examples

os.environ["ECMWF_API_KEY"] = 'c73ca7414cebdf3a6c12b86b3b84cabd'
os.environ["ECMWF_API_EMAIL"] = 'christopher.kalima@gmail.com'

server.retrieve({
    'origin'    : "kwbc",
    'levelist'  : "200",
    'levtype'   : "pl",
    'expver'    : "prod",
    'dataset'   : "tigge",
    'step'      : "0/6/12/18/24/30",
    'grid'      : "0.5/0.5",
    'param'     : "131/132",
    'time'      : "00/06/12/18",
    'date'      : "2014-10-01",
    'type'      : "cf",
    'class'     : "ti",
    'target'    : "tigge_2014-10-01_00061218.grib"
})

2016-11-01 13:02:34 ECMWF API python library 1.4.1
2016-11-01 13:02:34 ECMWF API at https://api.ecmwf.int/v1
2016-11-01 13:02:35 Welcome Chris Kalima


APIException: "ecmwf.API error 1: User 'christopher.kalima@gmail.com' has not access to datasets/tigge. Please accept the terms and conditions at http://apps.ecmwf.int/datasets/licences/tigge"

The API in this case raises an `APIException` and includes a link to the terms and conditions page that must be accepted by the user before the dataset can be accessed.